<h1>Capstone Project Week 3 - Peer Graded Assignment</h1><br>
Maps can't be rendered on Github. Therefore, use the link below to view maps<br>
https://nbviewer.jupyter.org/github/hithesh111/Coursera_Capstone/blob/master/Capstone_Submission_4.ipynb

In [1]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
# !pip install folium
import folium
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

<h3>Scraping the data from Wikipedia and getting it in the  form that we require.</h3>

In [2]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
neighborhood_data = data[0]
neighborhood_data = neighborhood_data[neighborhood_data['Borough']!='Not assigned']
neighborhood_data.index = range(neighborhood_data.shape[0])
neighborhood_data['Neighborhood'] = neighborhood_data['Neighborhood'].apply(lambda x: x.replace(' /',','))
neighborhood_data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3>Loading the geospatial coordinates csv file as a dataframe.</h3>
Hidden because it involves credentials.

In [3]:
# The code was removed by Watson Studio for sharing.

<h3>Adding Latitude and Longitude columns from the geospacial coordinates data</h3>

In [4]:
df_data_1.head()
df_data_1.set_index(['Postal Code'],inplace=True)

def get_lat_long(geo_df,code):
    lat = geo_df.loc[code,'Latitude']
    long = geo_df.loc[code,'Longitude']
    return(lat,long)

neighborhood_data['Latitude'] = neighborhood_data['Postal Code'].apply(lambda x: get_lat_long(df_data_1,x)[0])
neighborhood_data['Longitude'] = neighborhood_data['Postal Code'].apply(lambda x: get_lat_long(df_data_1,x)[1])
neighborhood_data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h3>Using only rows that have Borough names with Toronto in it.</h3>

In [5]:
east_toronto = neighborhood_data[neighborhood_data['Borough']=='East Toronto']
west_toronto = neighborhood_data[neighborhood_data['Borough']=='West Toronto']
central_toronto = neighborhood_data[neighborhood_data['Borough']=='Central Toronto']
downtown_toronto = neighborhood_data[neighborhood_data['Borough']=='Downtown Toronto']
neighborhood_data = pd.concat([east_toronto,west_toronto,central_toronto,downtown_toronto],axis=0)
neighborhood_data.index = range(neighborhood_data.shape[0])
neighborhood_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558


FourSquare API credentials

In [6]:
# @hidden cell
CLIENT_ID = 'KNV3TI3PR1F0QHFCOOEYL22Z4LEFCZ20GUPNKCIYROOCVMOQ'
CLIENT_SECRET = 'KYTRCRJLBVR2N0BBA4FAKK3CRNO3HSA2EHOMCLIJRV3BZNJX'
VERSION = '20180605'

<h3>The latitude is of toronto and we wish to find venues within 500km radius</h3>

In [7]:
RADIUS = 500
latitude = 43.6532
longitude = -79.3832
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&radius={}&ll={},{}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,RADIUS,latitude,longitude)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=KNV3TI3PR1F0QHFCOOEYL22Z4LEFCZ20GUPNKCIYROOCVMOQ&client_secret=KYTRCRJLBVR2N0BBA4FAKK3CRNO3HSA2EHOMCLIJRV3BZNJX&v=20180605&radius=500&ll=43.6532,-79.3832


In [8]:
results = requests.get(url).json()
# results

<h3>Function which returns the category of the venue using FourSquare API</h3>

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.shape)
nearby_venues.head()

(30, 4)


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,LUSH,Cosmetics Shop,43.653557,-79.380400
4,CF Toronto Eaton Centre,Shopping Mall,43.654540,-79.380677


<h3>Function which returns nearby venues</h3>

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION, lat, lng, radius)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],  v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

<h3>Displaying all Nearby Venues in Toronto</h3>

In [12]:
latitudes = neighborhood_data['Latitude']
longitudes = neighborhood_data['Longitude']

toronto_venues = getNearbyVenues(neighborhood_data['Neighborhood'],latitudes,longitudes)

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Business reply mail Processing Centre
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

<h3>Converting category column into one-hot features to use for KMeans clustering algorithm</h3>

In [13]:
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'])
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
# print(toronto_onehot.columns.tolist().index('Neighborhood'))
# print(len(toronto_onehot.columns.tolist()))
not_neigh_indices = list(range(133)) + list(range(134,189))
fixed_columns = [toronto_onehot.columns[133]] + list(toronto_onehot.columns[not_neigh_indices])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3>Grouped all venues in the one hot dataframe by neighborhood.</h3>

In [14]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.033333,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.043478
2,Business reply mail Processing Centre,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333


In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Making a dataframe of the most common categories of venues visited in a Neighborhood

In [16]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Park
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Restaurant
2,Business reply mail Processing Centre,Yoga Studio,Butcher,Farmers Market,Auto Workshop,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Harbor / Marina
4,Central Bay Street,Coffee Shop,Café,Yoga Studio,Bubble Tea Shop,Hotel


<h3>KMeans Clustering and fitted Labels. No of clusters are chosen to be five.</h3>

In [17]:
num_clusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',axis = 1)
kmeans = KMeans(num_clusters,random_state=42).fit(toronto_grouped_clustering)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 4, 0,
       0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Clubbing all the relevant data together

In [18]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhood_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Trail,Health Food Store,Pub,Colombian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Bakery,Coffee Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Fast Food Restaurant,Fish & Chips Shop,Brewery,Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Yoga Studio,Cheese Shop
4,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558,0,Yoga Studio,Butcher,Farmers Market,Auto Workshop,Burrito Place


In [19]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Clusters</h3>

In [20]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,The Beaches,0,Coffee Shop,Trail,Health Food Store,Pub,Colombian Restaurant
1,"The Danforth West, Riverdale",0,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Bakery,Coffee Shop
2,"India Bazaar, The Beaches West",0,Park,Fast Food Restaurant,Fish & Chips Shop,Brewery,Restaurant
3,Studio District,0,Café,Coffee Shop,Bakery,Yoga Studio,Cheese Shop
4,Business reply mail Processing Centre,0,Yoga Studio,Butcher,Farmers Market,Auto Workshop,Burrito Place
5,"Dufferin, Dovercourt Village",0,Pharmacy,Bakery,Gym / Fitness Center,Bank,Pool
6,"Little Portugal, Trinity",0,Bar,Vietnamese Restaurant,Asian Restaurant,Men's Store,Yoga Studio
7,"Brockton, Parkdale Village, Exhibition Place",0,Café,Breakfast Spot,Coffee Shop,Yoga Studio,Restaurant
8,"High Park, The Junction South",0,Thai Restaurant,Café,Mexican Restaurant,Bar,Bakery
9,"Parkdale, Roncesvalles",0,Gift Shop,Cuban Restaurant,Eastern European Restaurant,Breakfast Spot,Italian Restaurant


In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,Forest Hill North & West,1,Park,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio
34,Rosedale,1,Park,Playground,Trail,Cuban Restaurant,Donut Shop


In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,Roselawn,2,Home Service,Garden,Ice Cream Shop,Yoga Studio,Eastern European Restaurant


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,Lawrence Park,3,Park,Bus Line,Swim School,Eastern European Restaurant,Donut Shop


In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,"Moore Park, Summerhill East",4,Gym,Restaurant,Yoga Studio,Dance Studio,Donut Shop
